<a href="https://colab.research.google.com/github/coco401/deep-learning-models-in-tf2/blob/master/Lenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf
from tensorflow.keras import models,layers
import numpy as np


In [12]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


Layer C1 is a convolution layer with six convolution kernels of 5x5 and the size of feature mapping is 28x28, which can prevent the information of the input image from falling out of the boundary of convolution kernel.

Layer S2 is the subsampling/pooling layer that outputs 6 feature graphs of size 14x14. Each cell in each feature map is connected to 2x2 neighborhoods in the corresponding feature map in C1.

Layer C3 is a convolution layer with 16 5-5 convolution kernels. The input of the first six C3 feature maps is each continuous subset of the three feature maps in S2, the input of the next six feature maps comes from the input of the four continuous subsets, and the input of the next three feature maps comes from the four discontinuous subsets. Finally, the input for the last feature graph comes from all feature graphs of S2.

Layer S4 is similar to S2, with size of 2x2 and output of 16 5x5 feature graphs.

Layer C5 is a convolution layer with 120 convolution kernels of size 5x5. Each cell is connected to the 5*5 neighborhood on all 16 feature graphs of S4. Here, since the feature graph size of S4 is also 5x5, the output size of C5 is 1*1. So S4 and C5 are completely connected. C5 is labeled as a convolutional layer instead of a fully connected layer, because if lenet-5 input becomes larger and its structure remains unchanged, its output size will be greater than 1x1, i.e. not a fully connected layer.

F6 layer is fully connected to C5, and 84 feature graphs are output.

In [27]:
model = models.Sequential([
  Input(shape = input_shape),
  layers.Conv2D(6, 5,  activation='relu', padding = 'same'),
  layers.MaxPooling2D(),
  layers.Conv2D(16, 5, activation='relu',padding = "same"),
  layers.MaxPooling2D(), 
  layers.Conv2D(120, 5, activation='relu',padding = "same"),
  layers.Flatten(),
  layers.Dense(84, activation = 'tanh'),
  layers.Dense(num_classes,activation="softmax")
])
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_39 (Conv2D)           (None, 28, 28, 6)         156       
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 14, 14, 16)        2416      
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 7, 7, 16)          0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 7, 7, 120)         48120     
_________________________________________________________________
flatten_10 (Flatten)         (None, 5880)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 84)              

In [20]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
422/422 [==============================] - 19s 46ms/step - loss: 0.4539 - accuracy: 0.8752 - val_loss: 0.1287 - val_accuracy: 0.9650
Epoch 2/15
422/422 [==============================] - 19s 45ms/step - loss: 0.1251 - accuracy: 0.9636 - val_loss: 0.0817 - val_accuracy: 0.9785
Epoch 3/15
422/422 [==============================] - 19s 45ms/step - loss: 0.0935 - accuracy: 0.9721 - val_loss: 0.0724 - val_accuracy: 0.9818
Epoch 4/15
422/422 [==============================] - 19s 45ms/step - loss: 0.0772 - accuracy: 0.9765 - val_loss: 0.0616 - val_accuracy: 0.9823
Epoch 5/15
422/422 [==============================] - 19s 45ms/step - loss: 0.0678 - accuracy: 0.9796 - val_loss: 0.0573 - val_accuracy: 0.9848
Epoch 6/15
422/422 [==============================] - 20s 46ms/step - loss: 0.0599 - accuracy: 0.9811 - val_loss: 0.0530 - val_accuracy: 0.9847
Epoch 7/15
422/422 [==============================] - 20s 46ms/step - loss: 0.0538 - accuracy: 0.9838 - val_loss: 0.0512 - val_accuracy:

In [21]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.04211908206343651
Test accuracy: 0.9869999885559082
